In [9]:
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

# Name of the directory containing the object detection module we're using
MODEL_NAME = r'C:\Users\ajinf\Desktop\Swift\model'
PATH_TO_IMAGE = r'C:\Users\ajinf\Desktop\Swift\image.jpg'
im = Image.open(r'C:\Users\ajinf\Desktop\Swift\image.jpg')
# Grab path to current working directory
CWD_PATH = os.getcwd()
# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')
# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'data','labelmap.pbtxt')
# Number of classes the object detector can identify
NUM_CLASSES = 1

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    ograph = tf.Graph()
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.compat.v1.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
image = cv2.imread(PATH_TO_IMAGE)
image_expanded = np.expand_dims(image, axis=0)

# Perform the actual detection by running the model with the image as input
(boxes, scores, classes, num) = sess.run(
    [detection_boxes, detection_scores, detection_classes, num_detections],
    feed_dict={image_tensor: image_expanded})

# Draw the results of the detection (aka 'visulaize the results')
image, array_coord = vis_util.visualize_boxes_and_labels_on_image_array(
    image,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=3,
    min_score_thresh=0.60)

ymin, xmin, ymax, xmax = array_coord

shape = np.shape(image)
im_width, im_height = shape[1], shape[0]
(left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)

# Using Image to crop and save the extracted copied image
im.crop((left, top, right, bottom)).save(r'C:\Users\ajinf\Desktop\Swift\final.png', quality=95)
cropped_image = im.crop((left, top, right, bottom))

image_cropped = cv2.imread(r'C:\Users\ajinf\Desktop\Swift\final.png')
cv2.imshow("ID-CARD-CROPPED:", image_cropped)
# Press any key to close the image
cv2.waitKey(0)

# Clean up
cv2.destroyAllWindows()

In [10]:
import cv2
import pytesseract
import numpy as np
import matplotlib.pyplot as plt
import imutils
from numpy import asarray
#Declare pytessecart excuteable path

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
    
from PIL import Image

rot_img = asarray(cropped_image)
gray = cv2.cvtColor(rot_img, cv2.COLOR_BGR2GRAY)
ret, th1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU |
                                          cv2.THRESH_BINARY_INV)
cv2.imshow("OK",th1)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [11]:
data = pytesseract.image_to_string(th1, config='--oem 1 --psm 6')

In [12]:
IDCARD = ['REVA', 'UNIVERSITY', 'STUDENT', 'ID', 'CARD', 'VALIDITY', 'SRN', 'REGISTRAR', 'TO', 
          'NAME', 'COURSE', 'ADMISSION', 'NO', 'SIGNATURE','OF', 'ISSUING', 'AUTHORITY']

In [13]:
words = []
for i in IDCARD:
    if i in data:
        words.append(i)
        
if len(words) >= 3:
    print("Approved")
else:
    print("Not Approved, Upload Again")

Approved


In [19]:
def cropImageRoi(image, roi):
    roi_cropped = image[
        int(roi[1]) : int(roi[1] + roi[3]), int(roi[0]) : int(roi[0] + roi[2])
    ]
    return roi_cropped

class ImageConstantROI():
    class CCCD(object):
        ROIS = {
            "id": [(280, 100, 500, 400)],
            "name": [
                (325, 393, 771, 64),
            ],
            "birth_date": [(658, 451, 216, 54)],
            "gender": [(538, 495, 95, 44)],
            "address": [(786, 639, 343, 39), (318, 672, 791, 52)],
            "place_birth": [(326, 586, 792, 47)],
            "date_expire": [(155, 653, 152, 31)],
        }

        CHECK_ROI = [(313, 174, 597, 63)]

testCrop = cropImageRoi(rot_img, ImageConstantROI.CCCD.ROIS['id'][0])




In [14]:
data

'| =, REVA_ | cue\n| SS UNIVERSITY | oo ld\nil ak ee\nal Ge R21SL013 AE\n{Bees Be\nByemanTHRA sO\n¥ |*.Sc in Bio-Technology ae\nae | REGISTRAR Ji\n'

In [15]:
words

['REVA', 'UNIVERSITY', 'REGISTRAR']

In [21]:
cv2.imshow("ID-CARD-CROPPED:", testCrop)
# Press any key to close the image
cv2.waitKey(0)

# Clean up
cv2.destroyAllWindows()